In [ ]:
%matplotlib inline

import math
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import requests
from io import BytesIO
import base64, os

##############################################################################
# 0. CSS EXTRA PARA DEIXAR O "x" EM VERMELHO E SEM DESTAQUE
##############################################################################
display(HTML('''
<style>
.btn-red-text > button {
    color: red !important;             /* cor da fonte em vermelho */
    background-color: white !important; /* fundo branco */
    border: 1px solid #ccc !important;  /* borda leve */
    border-radius: 4px !important;      /* cantos levemente arredondados */
    padding: 0 5px !important;          /* espaço horizontal pequeno */
    font-weight: bold !important;       /* negrito */
}
</style>
'''))

##############################################################################
# 1. DADOS INICIAIS
##############################################################################
# Notas de Execução e Visão começam como None para ficar "em branco" no widget.
concorrentes_data = [
    ["FluencyPass",     None, None, "https://i.ibb.co/NVbyMKD/fluency-pass.png"],
    ["Fluency Academy", None, None, "https://i.ibb.co/7tRhz0r/fluency-academy.png"],
    ["Open English",    None, None, "https://i.ibb.co/stC9gr6/open-english.png"],
    ["GoFluent",        None, None, "https://i.ibb.co/Dkqf08Y/gofluent.png"],
    ["Rosetta",         None, None, "https://i.ibb.co/HqDKqzg/rosetta.png"],
    ["Voxy",            None, None, "https://i.ibb.co/bbXm5vN/voxy.png"],
    ["Nulinga",         None, None, "https://i.ibb.co/xSbgHzW/nulinga.png"],
    ["Berlitz",         None, None, "https://i.ibb.co/y0TmqmS/berlitz.png"],
    ["Yázigi",          None, None, "https://i.ibb.co/wM1n1Bh/yazigi.png"],
    ["Flexge",          None, None, "https://i.ibb.co/cXtpCqy/flexge.png"],
    ["Preply",          None, None, "https://i.ibb.co/WDJtvmK/preply.png"],
    ["English Live",    None, None, "https://i.ibb.co/8gyt7HK/english-live.png"],
]

##############################################################################
# 2. FUNÇÕES AUXILIARES
##############################################################################
def definir_cor(visao, execucao):
    """Define a cor do marcador com base no quadrante (visão vs. execução)."""
    if visao > 3 and execucao > 3:
        return "green"   # Líderes
    elif visao <= 3 and execucao > 3:
        return "blue"    # Visionários
    elif visao > 3 and execucao <= 3:
        return "orange"  # Desafiantes
    else:
        return "red"     # Nichos

def carregar_imagem(url_logo, zoom=0.20):
    """Carrega a imagem de um URL e retorna um OffsetImage para plotagem."""
    if not url_logo:
        return None
    try:
        response = requests.get(url_logo)
        response.raise_for_status()
        img_data = BytesIO(response.content)
        return OffsetImage(plt.imread(img_data, format='png'), zoom=zoom)
    except Exception as e:
        print(f"Erro ao carregar imagem: {e}")
        return None

def parse_float(s):
    """
    Converte string em float, aceitando vírgula ou ponto.
    Ex.: "3,4" -> 3.4
    """
    s = s.strip().replace(',', '.')
    return float(s)

##############################################################################
# 2.1. VARIÁVEL GLOBAL PARA GUARDAR A FIGURA (PARA SALVAR COMO IMAGEM)
##############################################################################
last_fig = None

##############################################################################
# 3. CRIAÇÃO DA TABELA (TELA DE PREENCHIMENTO)
##############################################################################
def criar_cabecalho():
    """Cria o cabeçalho da tabela (5 colunas)."""
    return widgets.HBox([
        widgets.HTML("<b>Concorrente</b>", layout=widgets.Layout(width='150px')),
        widgets.HTML("<b>Nota Execução</b>", layout=widgets.Layout(width='120px')),
        widgets.HTML("<b>Nota Visão</b>", layout=widgets.Layout(width='120px')),
        widgets.HTML("<b>URL Logo</b>", layout=widgets.Layout(width='280px')),
        widgets.HTML("", layout=widgets.Layout(width='40px')),  # Coluna do "x"
    ])

def criar_linha_concorrente(nome, exec_val, visao_val, url_val):
    nome_widget = widgets.Text(
        value=nome if nome else "",
        placeholder='Nome do concorrente',
        layout=widgets.Layout(width='150px')
    )

    exec_widget = widgets.FloatText(
        value=exec_val,
        placeholder='Insira a nota (1-5)',
        layout=widgets.Layout(width='120px')
    )
    visao_widget = widgets.FloatText(
        value=visao_val,
        placeholder='Insira a nota (1-5)',
        layout=widgets.Layout(width='120px')
    )

    url_widget = widgets.Text(
        value=url_val if url_val else "",
        layout=widgets.Layout(width='280px')
    )

    # Botão "x" em vermelho
    botao_excluir = widgets.Button(
        description="x",
        button_style='',
        layout=widgets.Layout(width='30px', height='30px', padding='0px')
    )
    botao_excluir.add_class("btn-red-text")

    linha = widgets.HBox([
        nome_widget, exec_widget, visao_widget, url_widget, botao_excluir
    ])

    linha_dict = {
        "container": linha,
        "nome": nome_widget,
        "exec": exec_widget,
        "visao": visao_widget,
        "url": url_widget,
        "excluir": botao_excluir
    }

    return linha, linha_dict

linhas_widgets = []
cabecalho = criar_cabecalho()
tabela = widgets.VBox()

def montar_tabela_inicial():
    linhas_widgets.clear()
    linhas_hbox = [cabecalho]

    for nome, exec_val, visao_val, url_val in concorrentes_data:
        linha, linha_dict = criar_linha_concorrente(nome, exec_val, visao_val, url_val)

        def on_excluir_clicked(b, row_dict=linha_dict):
            linhas_widgets.remove(row_dict)
            atualizar_tabela()

        linha_dict["excluir"].on_click(on_excluir_clicked)
        linhas_widgets.append(linha_dict)
        linhas_hbox.append(linha)

    tabela.children = linhas_hbox

def atualizar_tabela():
    linhas_hbox = [cabecalho]
    for row_dict in linhas_widgets:
        linhas_hbox.append(row_dict["container"])
    tabela.children = linhas_hbox

montar_tabela_inicial()

##############################################################################
# 4. BOTÕES PRINCIPAIS (ADICIONAR, GERAR GRÁFICO) + COLAR NOTAS
##############################################################################
botao_adicionar = widgets.Button(
    description="Adicionar concorrente",
    button_style='info'
)
botao_gerar_grafico = widgets.Button(
    description="Gerar gráfico",
    button_style='success'
)

botao_colar_notas = widgets.Button(
    description="Colar Notas (Exec, Visão)",
    button_style='',
    tooltip="Copie duas colunas (Execução, Visão) do Excel e cole aqui"
)

paste_area = widgets.Textarea(
    placeholder=(
        "Cole aqui o texto no formato:\n"
        "Execução\tVisão\n"
        "3,4\t3,0\n"
        "3,2\t3,0\n"
        "... etc.\n"
        "Depois clique em Aplicar colagem."
    ),
    layout=widgets.Layout(width='400px', height='150px', display='none')
)

botao_aplicar_colagem = widgets.Button(
    description="Aplicar colagem",
    button_style='info',
    layout=widgets.Layout(display='none')
)

def on_toggle_colar_notas(b):
    if paste_area.layout.display == 'none':
        paste_area.layout.display = 'block'
        botao_aplicar_colagem.layout.display = 'inline-block'
    else:
        paste_area.layout.display = 'none'
        botao_aplicar_colagem.layout.display = 'none'

botao_colar_notas.on_click(on_toggle_colar_notas)

def on_aplicar_colagem_clicked(b):
    linhas = paste_area.value.strip().split('\n')
    idx = 0
    for linha in linhas:
        if idx >= len(linhas_widgets):
            break
        temp = linha.replace(';', '\t')
        parts = [p for p in temp.split() if p]

        if len(parts) == 2 and ("xecu" in parts[0] or "Vis" in parts[1]):
            continue

        if len(parts) < 2:
            continue

        try:
            exec_val = parse_float(parts[0])
            visao_val = parse_float(parts[1])
            linhas_widgets[idx]["exec"].value = exec_val
            linhas_widgets[idx]["visao"].value = visao_val
            idx += 1
        except:
            pass

    paste_area.layout.display = 'none'
    botao_aplicar_colagem.layout.display = 'none'

botao_aplicar_colagem.on_click(on_aplicar_colagem_clicked)

##############################################################################
# 6. FUNÇÃO "ADICIONAR CONCORRENTE"
##############################################################################
def on_adicionar_clicked(b):
    linha, linha_dict = criar_linha_concorrente("Novo Concorrente", None, None, "")
    def on_excluir_clicked(b, row_dict=linha_dict):
        linhas_widgets.remove(row_dict)
        atualizar_tabela()

    linha_dict["excluir"].on_click(on_excluir_clicked)
    linhas_widgets.append(linha_dict)
    atualizar_tabela()

botao_adicionar.on_click(on_adicionar_clicked)

##############################################################################
# 7. TELA DO GRÁFICO E BOTÃO VOLTAR
##############################################################################
output_plot = widgets.Output()
botao_voltar = widgets.Button(
    description="Voltar",
    button_style='warning'
)

tela_grafico = widgets.VBox([
    output_plot,
    botao_voltar
])
tela_grafico.layout.display = 'none'

##############################################################################
# 8. MOSTRAR/ESCONDER TELAS
##############################################################################
tela_preenchimento = widgets.VBox([
    widgets.HBox([botao_gerar_grafico, botao_adicionar, botao_colar_notas]),
    widgets.HBox([paste_area, botao_aplicar_colagem]),
    tabela
])

def mostrar_tela_preenchimento():
    tela_preenchimento.layout.display = 'block'
    tela_grafico.layout.display = 'none'

def mostrar_tela_grafico():
    tela_preenchimento.layout.display = 'none'
    tela_grafico.layout.display = 'block'

##############################################################################
# 9. GERAR O GRÁFICO (VALIDANDO 1..5) + SALVAR FIGURA EM 'last_fig'
##############################################################################
def on_gerar_grafico_clicked(b):
    global last_fig
    with output_plot:
        clear_output(wait=True)

        dados_atualizados = []
        for row_dict in linhas_widgets:
            nome = row_dict["nome"].value
            exec_val = row_dict["exec"].value
            visao_val = row_dict["visao"].value
            url_val = row_dict["url"].value

            if exec_val is None or visao_val is None:
                continue
            if not (1 <= exec_val <= 5) or not (1 <= visao_val <= 5):
                continue

            dados_atualizados.append([nome, exec_val, visao_val, url_val])

        if not dados_atualizados:
            print("⚠️ Nenhum concorrente com notas válidas para exibir no gráfico (1 a 5).")
            return

        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(1, 5)
        ax.set_ylim(1, 5)

        ax.axhline(y=3, color="gray", linestyle="--", linewidth=1)
        ax.axvline(x=3, color="gray", linestyle="--", linewidth=1)

        ticks = [i * 0.5 for i in range(2, 11)]
        ax.set_xticks(ticks)
        ax.set_yticks(ticks)
        ax.set_xticklabels(ticks, fontsize=10)
        ax.set_yticklabels(ticks, fontsize=10)

        ax.set_xlabel("Visão", fontsize=12)
        ax.set_ylabel("Execução", fontsize=12)
        ax.set_facecolor("white")
        ax.grid(True, color='lightgray', linestyle='-', linewidth=0.5)

        for nome, exec_val, visao_val, url_logo in dados_atualizados:
            cor = definir_cor(visao_val, exec_val)
            if url_logo:
                logo_img = carregar_imagem(url_logo, zoom=0.20)
                if logo_img:
                    ab = AnnotationBbox(logo_img, (visao_val, exec_val), frameon=False)
                    ax.add_artist(ab)
                else:
                    ax.plot(visao_val, exec_val, marker='o', color=cor, markersize=10)
            else:
                ax.plot(visao_val, exec_val, marker='o', color=cor, markersize=10)

        plt.tight_layout()
        plt.show()
    
    # Guarda a figura gerada em 'last_fig'
    last_fig = fig
    mostrar_tela_grafico()

def on_voltar_clicked(b):
    mostrar_tela_preenchimento()

botao_gerar_grafico.on_click(on_gerar_grafico_clicked)
botao_voltar.on_click(on_voltar_clicked)

##############################################################################
# 9.1. BOTÃO PARA SALVAR A IMAGEM
##############################################################################
botao_salvar_imagem = widgets.Button(
    description="Salvar imagem",
    button_style='info'
)
saida_salvar = widgets.Output()

def on_salvar_imagem_clicked(b):
    global last_fig
    with saida_salvar:
        clear_output()
        if last_fig is None:
            print("Não há figura para salvar.")
            return
        
        # Salvar localmente
        last_fig.savefig("grafico.png")
        
        # Converter em base64
        with open("grafico.png", "rb") as f:
            b64 = base64.b64encode(f.read()).decode('utf-8')
        
        # Remover o arquivo local (opcional)
        os.remove("grafico.png")
        
        # Criar link de download
        link = f'<a download="grafico.png" href="data:image/png;base64,{b64}">Clique aqui para baixar o PNG</a>'
        display(HTML(link))

botao_salvar_imagem.on_click(on_salvar_imagem_clicked)

##############################################################################
# 10. EXIBIR A INTERFACE FINAL (AGORA COM O BOTÃO SALVAR)
##############################################################################
# Vamos adicionar o botao_salvar_imagem e saida_salvar na tela do gráfico,
# ao lado do botao_voltar, por exemplo.
tela_grafico.children = [
    output_plot,
    widgets.HBox([botao_voltar, botao_salvar_imagem]),
    saida_salvar
]

mostrar_tela_preenchimento()
display(tela_preenchimento, tela_grafico)
